Alunos:

*   Andréa Fonseca
*   Fábio Cardoso
*   Eduardo Leite



In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Modelos

In [2]:
!pip install langchain_openai langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 593.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Succes

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
response = llm.invoke("Olá")

print(response)

content='Olá! Como posso ajudar você hoje?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 8, 'total_tokens': 16}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None} id='run-11c81603-9932-4cc1-bc5b-0c347efeb461-0' usage_metadata={'input_tokens': 8, 'output_tokens': 8, 'total_tokens': 16}


In [5]:
response.content

'Olá! Como posso ajudar você hoje?'

## Prompts

### Templates Simples

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("Traduza o seguinte texto para português: {texto}")

#runnable
prompt = prompt_template.invoke({"texto": "He would have gone to the gym, if it was not raining."})
print(prompt)

messages=[HumanMessage(content='Traduza o seguinte texto para português: He would have gone to the gym, if it was not raining.')]


In [7]:
response = llm.invoke(prompt)

print(response.content)

Ele teria ido à academia, se não estivesse chovendo.


### Templates de Mensagens

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
    HumanMessage(content="Hello, how are you?"),
]

# messages = [
#     ("system", "Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
#     ("human", "Hello, how are you?"),
# ]

response = llm.invoke(messages)

print(response)

content='Olá, como você está?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 36, 'total_tokens': 42}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None} id='run-3820a454-05a9-4eea-8766-d232d55d0d7f-0' usage_metadata={'input_tokens': 36, 'output_tokens': 6, 'total_tokens': 42}


In [9]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor de {lingua_origem} para {lingua_destino}. Traduza as mensagens que forem enviadas."),
        ("user", "{texto}")
    ]
)

In [10]:
prompt = prompt_template.invoke({
    "lingua_origem": "alemão",
    "lingua_destino": "português",
    "texto": "Wie gehts?"
})

print(prompt)

messages=[SystemMessage(content='Você é um tradutor de alemão para português. Traduza as mensagens que forem enviadas.'), HumanMessage(content='Wie gehts?')]


In [11]:
response = llm.invoke(prompt)

print(response.content)

Como você está?


### Parsers

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [13]:
output = parser.invoke(response)

print(output)

Como você está?


## Encadeamento

In [14]:
chain = prompt_template | llm | parser

In [15]:
response = chain.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "espanhol",
    "texto": "As praias de Recife tem tubarões!"
})

print(response)

¡Las playas de Recife tienen tiburones!


In [16]:
def translate(texto, lingua_origem, lingua_destino):
    response = chain.invoke({
        "lingua_origem": lingua_origem,
        "lingua_destino": lingua_destino,
        "texto": texto
    })
    return response

In [17]:
output = translate("The beaches of Recife have sharks!", "inglês", "espanhol")

print(output)

¡Las playas de Recife tienen tiburones!


## Exercícios

### Exercício 1
Crie uma `chain` que a partir de um tópico informado pelo usuário, crie uma piada.

In [18]:
prompt_template = ChatPromptTemplate.from_template("Crie uma piada sobre {topic}")

chain = prompt_template | llm | parser

chain.invoke({
    "topic": "futebol"
})



'Por que o jogador de futebol foi ao banco?\n\nPorque ele queria melhorar sua "conta" de gols! ⚽💰'

### Exercício 2
Crie uma `chain` que classifique o sentimento de um texto de entrada em positivo, neutro ou negativo.

In [19]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você deverá classificar o sentimento de um {texto_entrada} em positivo, neutro ou negativo. Classifique os textos enviados."),
        ("user", "{texto_entrada}")
    ]
)

In [20]:
chain = prompt_template | llm | parser

chain.invoke({
    "texto_entrada": "há 23 anos o mundo perdia Aaliyah. A cantora estava voltando das gravações de um clipe quando o avião no qual ela estava viajando caiu nas Bahamas logo após a decolagem, matando todas as 9 pessoas a bordo."
})


'Negativo'

### Exercício 3
Crie uma `chain` que gere o código de uma função Python de acordo com a descrição do usuário.

In [21]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um programador de Python deve gerar o código de uma função de acordo com a {descricao} do usuário. Gere o código Python conforme a descrição enviada."),
        ("user", "{descricao}")
    ]
)

In [23]:
chain = prompt_template | llm | parser

response = chain.invoke({
    "descricao": "contar palavras em um texto",
})

In [24]:
print(response)

Aqui está uma função em Python que conta o número de palavras em um texto fornecido pelo usuário:

```python
def contar_palavras(texto):
    # Remove espaços em branco extras e divide o texto em palavras
    palavras = texto.strip().split()
    # Retorna o número de palavras
    return len(palavras)

# Solicita ao usuário para inserir um texto
texto_usuario = input("Digite um texto: ")
# Conta as palavras no texto e exibe o resultado
numero_palavras = contar_palavras(texto_usuario)
print(f"O número de palavras no texto é: {numero_palavras}")
```

### Como funciona:
1. A função `contar_palavras` aceita uma string como argumento.
2. Usa `strip()` para remover espaços em branco no início e no final da string.
3. Usa `split()` para dividir o texto em uma lista de palavras, com base em espaços.
4. Retorna o comprimento da lista, que representa o número de palavras.
5. O programa solicita que o usuário insira um texto e, em seguida, exibe o número de palavras contadas. 

Você pode executar e

### Exercício 4
Crie uma `chain` que explique de forma simplificada um tópico geral fornecido pelo usuário e, em seguida, traduza a explicação para inglês. Utilize dois templates encadeados.

In [25]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você deverá explicar de forma simplificada um {topico_geral} fornecido pelo usuário e, em seguida, traduzir a explicação para inglês."),
        ("user", "{topico_geral}")
    ]
)

In [26]:
chain = prompt_template | llm | parser

response = chain.invoke({
    "topico_geral": "queimadas pelo Brasil",
})

In [27]:
print(response)

As queimadas no Brasil referem-se ao ato de colocar fogo em áreas de vegetação, muitas vezes para limpar terrenos para agricultura ou pastagens. Esse processo pode causar grandes danos ao meio ambiente, incluindo a destruição de habitats, a liberação de gases poluentes e o aumento da temperatura do clima. As queimadas costumam ocorrer principalmente na Amazônia, mas também em outras regiões do país. O controle e a prevenção das queimadas são importantes para proteger a biodiversidade e combater as mudanças climáticas.

---

Burning in Brazil refers to the act of setting fire to areas of vegetation, often to clear land for agriculture or pastures. This process can cause significant harm to the environment, including the destruction of habitats, the release of polluting gases, and an increase in climate temperature. Burnings usually occur mainly in the Amazon, but also in other regions of the country. Controlling and preventing burnings is important to protect biodiversity and combat cli

### Exercício 5 - Desafio
Crie uma `chain` que responda perguntas sobre o CESAR School.

In [28]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você deverá responder a uma {pergunta} do usuário sobre o CESAR School, como se fosse funcionário do CESAR School. Seja cortez."),
        ("user", "{pergunta}")
    ]
)

In [29]:
chain = prompt_template | llm | parser

response = chain.invoke({
    "pergunta": "Como faço para consultar minhas notas no curso de Pós-Graduação?",
})

In [30]:
print(response)

Olá! Para consultar suas notas no curso de Pós-Graduação da CESAR School, você pode acessar o portal do aluno através do nosso site oficial. Uma vez logado, você encontrará a seção de "Notas" ou "Desempenho Acadêmico", onde poderá visualizar suas avaliações.

Caso tenha algum problema para acessar sua conta ou precise de mais informações, sinta-se à vontade para entrar em contato com nossa equipe de suporte ao aluno pelo e-mail ou telefone disponíveis no site. Estamos aqui para ajudar!
